In [1]:
import requests
from pprint import pprint
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

from tqdm.notebook import tqdm


# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

D:\ANACONDA3\envs\py38-env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
params = {'apikey' : '더미',
          'sdCd' : 48,
          'sggCd' : 48250 }

response = requests.get(f'https://stcis.go.kr/openapi/areacode.json',params = params)
response

<Response [200]>

In [3]:
b = response.json()
b

{'count': 108,
 'status': 'OK',
 'result': [{'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825036026',
   'emdNm': '상동면'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825037000',
   'emdNm': '대동면'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825034032',
   'emdNm': '한림면'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825013200',
   'emdNm': '장유동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825032022',
   'emdNm': '주촌면'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825035023',
   'emdNm': '생림면'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825011400',
   'emdNm': '화목동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825025000',
  

In [8]:
len(b['result'])

108

In [4]:
b['result'][0]['emdCd']

'4825036026'

In [14]:
params = {'apikey' : '더미',
          'opratDate' : '20210320',
          'stgEmdCd' : s_emd['emdCd'],
          'arrEmdCd' : e_emd['emdCd']}
response = requests.get('https://stcis.go.kr/openapi/quarterod.json', 
                        params=params)

a = response.json()
a
#df = pd.DataFrame(a['result'])

{'status': 'NOT_FOUND'}

In [6]:
s_emd = b['result'][1]
e_emd = b['result'][7]


print('시작 : ' + s_emd['emdNm'])
print('끝 : ' + e_emd['emdNm'])


concat_df = None

for n in tqdm(range(7)): 
    params = {'apikey' : '더미',
              'opratDate' : f'2022012{n}',
              'stgEmdCd' : s_emd['emdCd'],
              'arrEmdCd' : e_emd['emdCd']}
    response = requests.get('https://stcis.go.kr/openapi/quarterod.json', 
                            params=params)
    
    a = response.json()
    df = pd.DataFrame(a['result'])
    
    if n == 0:
        concat_df = df
        continue
    concat_df = pd.concat([concat_df, df])

시작 : 대동면
끝 : 진영읍


  0%|          | 0/7 [00:00<?, ?it/s]

KeyError: 'result'

In [220]:
concat_df.reset_index(drop=True, inplace=True)

In [221]:
concat_df.columns

Index(['opratDate', 'stgSdCd', 'stgSdNm', 'stgSggCd', 'stgSggNm', 'stgEmdCd',
       'stgEmdNm', 'arrSdCd', 'arrSdNm', 'arrSggCd', 'arrSggNm', 'arrEmdCd',
       'arrEmdNm', 'tzon', 'quater', 'useStf', 'useTm'],
      dtype='object')

In [223]:
concat_df = concat_df.groupby(['opratDate', 'stgSdCd', 'stgSdNm', 'stgSggCd', 'stgSggNm', 'stgEmdCd',
       'stgEmdNm', 'arrSdCd', 'arrSdNm', 'arrSggCd', 'arrSggNm', 'arrEmdCd',
       'arrEmdNm'])['useStf', 'useTm'].sum()
concat_df

useStf  \
opratDate stgSdCd stgSdNm stgSggCd stgSggNm stgEmdCd   stgEmdNm arrSdCd arrSdNm arrSggCd arrSggNm arrEmdCd   arrEmdNm           
20220120  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            26   
20220121  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            19   
20220122  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            29   
20220123  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            16   
20220124  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            23   
20220125  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            20   
20220126  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동            20   

                                                                                                                       useTm  
opratDate stgSdCd stgSdNm stgSggCd stgSggNm stgEmdCd   stgEmdNm arrSdCd arrSdNm arrSggCd arrSggNm arrEmdCd   arrEmdNm         
20220120  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        10046  
20220121  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        10548  
20220122  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        16288  
20220123  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동         6450  
20220124  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        15104  
20220125  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        10499  
20220126  48      경상남도    48250    김해시      4825010200 서상동      48      경상남도    48250    김해시      4825010800 내동        15380

In [224]:
concat_df.to_csv('./서상동_내동.csv', encoding='cp949')

In [200]:
response

<Response [200]>

In [201]:
a = response.json()

In [202]:
a

{'count': 2,
 'result': [{'opratDate': '20220120',
   'stgSdCd': '48',
   'stgSdNm': '경상남도',
   'stgSggCd': '48250',
   'stgSggNm': '김해시',
   'stgEmdCd': '4825010100',
   'stgEmdNm': '동상동',
   'arrSdCd': '48',
   'arrSdNm': '경상남도',
   'arrSggCd': '48250',
   'arrSggNm': '김해시',
   'arrEmdCd': '4825010200',
   'arrEmdNm': '서상동',
   'tzon': '14',
   'quater': '00',
   'useStf': 1,
   'useTm': 297},
  {'opratDate': '20220120',
   'stgSdCd': '48',
   'stgSdNm': '경상남도',
   'stgSggCd': '48250',
   'stgSggNm': '김해시',
   'stgEmdCd': '4825010100',
   'stgEmdNm': '동상동',
   'arrSdCd': '48',
   'arrSdNm': '경상남도',
   'arrSggCd': '48250',
   'arrSggNm': '김해시',
   'arrEmdCd': '4825010200',
   'arrEmdNm': '서상동',
   'tzon': '15',
   'quater': '45',
   'useStf': 1,
   'useTm': 726}],
 'status': 'OK'}

In [204]:
pd.DataFrame(a['result'])

opratDate stgSdCd stgSdNm stgSggCd stgSggNm    stgEmdCd stgEmdNm arrSdCd  \
0  20220120      48    경상남도    48250      김해시  4825010100      동상동      48   
1  20220120      48    경상남도    48250      김해시  4825010100      동상동      48   

  arrSdNm arrSggCd arrSggNm    arrEmdCd arrEmdNm tzon quater  useStf  useTm  
0    경상남도    48250      김해시  4825010200      서상동   14     00       1    297  
1    경상남도    48250      김해시  4825010200      서상동   15     45       1    726

In [150]:
apikey = '더미'
opratDate = 20230131
stgEmdCd = 4825010100
arrEmdCd = 4825011000


In [131]:
response = requests.get(f'https://stcis.go.kr/openapi/quarterod.json?apikey={apikey}&opratDate={opratDate}&stgEmdCd={stgEmdCd}&arrEmdCd={arrEmdCd}')

In [132]:
response

<Response [200]>

In [133]:
a = response.json()
a

{'status': 'NOT_FOUND'}

In [84]:
response = requests.get(f'https://stcis.go.kr/openapi/areacode.json?apikey={apikey}&sdCd=48&sggCd=48250')
response

<Response [200]>

In [85]:
a = response.json()
a

{'count': 108,
 'status': 'OK',
 'result': [{'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010100',
   'emdNm': '동상동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010200',
   'emdNm': '서상동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010300',
   'emdNm': '부원동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010400',
   'emdNm': '봉황동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010500',
   'emdNm': '대성동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010600',
   'emdNm': '구산동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010700',
   'emdNm': '삼계동'},
  {'sdCd': '48',
   'sdNm': '경상남도',
   'sggCd': '48250',
   'sggNm': '김해시',
   'emdCd': '4825010800',
  